In [1]:
%load_ext autoreload
%autoreload 2
from fun import *
os.environ['QT_QPA_PLATFORM']='offscreen'

### AKAP79
* We use as reference the AKAP79 sequence in human
- **Gene:** AKAP5
- **Protein name:** AKAP79, AKAP5 ,…
- **Binding partners:** PKC,PKA, CaM, PP2B,…

#### Questions we want to address:

- Is the AKAP79 protein architecture the same in all species?
- How can we find the binding partners in the sequences? 
    - As a first step, by assesing the conservation of the msa, i.e., looking that the aligned sequences match with the reference sequence region. 
- Can we find repeated instances?
    - We can use a profile hmm for identifying binding regions and possible repetitions.
    
Disadvantages:
- One cannot conclude that a binding region is present in a species solely by looking at a msa
- the profile hmm might fail to identify binding regions, specially when few seed sequences are used for creating the model.

In [2]:
seq_filename = 'fasta_files/akap5_seq_prot.fa'
msa_filename = 'fasta_files/msa/akap5_seq_align.fa'
AKAP79_model_species = binding_regions()
AKAP79_model_species.muscle_msa(seq_filename, msa_filename)
# Parse msa and original sequences
akap5_seqs = AKAP79_model_species.parse_fasta_file(seq_filename)
akap5_msa = AKAP79_model_species.parse_fasta_file(msa_filename)

host_guest_dict = species_host_to_guest_map_dic()
nwk_direct = 'nwk_trees/model_species.nwk'

### Binding partners:

### 1. Calmodulin (CaM) - WSK domain - [Patel et al. Nature (2017)](https://www.nature.com/articles/s41467-017-01715-w)

#### msa of the reference binding region

In [3]:
# We will use as a reference the human architecture
ref = 'Homo_sapiens'
# calmodulin binding region - based on Patel et al.
base_string_wsk = 'RGAWASLKRLVTRRKRSESSKQQKPLEGEM'
start, end = AKAP79_model_species.find_binding_region(base_string_wsk, ref, akap5_msa)
# this binding region has a motif - its importance lies on its conservation
start_m, end_m = AKAP79_model_species.find_binding_region('WASLK', ref, akap5_msa)
wsk_residues_idx = (start_m,start_m+2,end_m-1)
filename = 'fasta_files/binding_regions/AKAP5_WSK.fa'
regions_dict, aln = seq_domain_alignment(akap5_msa,
                                         akap5_seqs,
                                         start, end,
                                         filename,
                                         motif_coords=wsk_residues_idx)

In [4]:
alv.view(aln)

Xenopus_tr GKTWATFKRLVTHKKKRHSS--LKQQSQMNSQ-
Ornithorhy QGAWAAIKHLVPHRKRSSSS--SKKQRCSEATE
Monodelphi GGTWASIRRLITRQKR-SKS--SKKQVPLDAKA
Mus_muscul -GAWASIKGLVTHRKR-SEP--AKKQKPPEAEV
Homo_sapie RGAWASLKRLVTRRKR-SES--SKQQKPLEGEM
Canis_lupu GGAWASIKRLVTRRKR-SES--SKQQKLFEAKV
Bos_taurus GGAWDSIKRLVTRRKR-SES--SKQQKPFKAKL
Sus_scrofa GGAWVSVKRLITCRKR-SDS--SKQQKPCKAKV
Anolis_car GGAWLAFKRLVTSRRR-SKSVLKKQQQSGGSRV
Gallus_gal RGAWAALKSLTKPRRGQKSS--SRKKVSSDSQV
Taeniopygi KGTWAAIKSLARPQRRQKSS--SRKKVPSDSQV
          0↑                 20↑


In general the msa seems quite well conserved. There is a variation in the second residue of the motif in which serine (S) is replaced by alanine (A). This occurs for birds, lizard and platypus.
- Is this an assembly artefact?
- What can these variations indicate?

#### Construct a hmm for identifying *calmodulin* binding regions in our model species

In [5]:
hmm_for_wsk_domain = binding_regions()
msa_filename = 'fasta_files/msa/PF03832_seed_msa.fa'
# align the seed sequences - downloaded from Pfam
hmm_for_wsk_domain.muscle_msa('profile_hmm/seed_files/PF03832_seed.fa', msa_filename)
# build the profile hmm
hmm_for_wsk_domain.build_profile_hmm("profile_hmm/hmm/WSK.hmm", msa_filename)
# search for the binding region
hmm_for_wsk_domain.search_binding_regions("profile_hmm/WSK.sto","profile_hmm/hmm/WSK.hmm",seq_filename)
wsk_hmm_hits = read_sto_files("profile_hmm/WSK.sto")

We want to know:
- How many binding regions (if any) the hmm finds per specie.
- Do the regions found coincide (or overlap) with the regions aligned to the reference sequence in the msa?

In [6]:
wsk_hits_counter = dict(Counter([specie_name.rsplit('-')[0] for specie_name in wsk_hmm_hits.keys()]))
wsk_aling_coord = {specie:list(regions_dict[specie].values())[0][0] for specie in regions_dict.keys()}
hmm_hits_analysis_df(regions_dict, wsk_hits_counter, wsk_aling_coord, wsk_hmm_hits)

,specie,number of instances,aligned to ref seq in msa
0,Gallus_gallus,0,-
1,Taeniopygia_guttata,0,-
2,Xenopus_tropicalis,1,True
3,Ornithorhynchus_anatinus,1,True
4,Monodelphis_domestica,1,True
5,Mus_musculus,1,True
6,Homo_sapiens,1,True
7,Canis_lupus,1,True
8,Bos_taurus,1,True
9,Sus_scrofa,1,True


No calmoduling binding regions were found in birds. We will improve the seed file with the hmm hits and check if we can find binding sites in the species where they were not found.

In [7]:
new_seeds = {}
for s in wsk_hmm_hits.keys():
    s = s.rsplit('-')[0]
    new_seeds[s] = akap5_seqs[s][wsk_aling_coord[s][0]:wsk_aling_coord[s][1]]

hmm_for_wsk_domain.generate_seed_file('profile_hmm/seed_files/PF03832_seed.fa',
                                      new_seeds,
                                      'profile_hmm/seed_files/PF03832_seed_v2.fa')

In [8]:
hmm_for_wsk_domain = binding_regions()
msa_filename = 'fasta_files/msa/PF03832_seed_v2_msa.fa'
# align the seed sequences - downloaded from Pfam
hmm_for_wsk_domain.muscle_msa('profile_hmm/seed_files/PF03832_seed_v2.fa', msa_filename)
# build the profile hmm
hmm_for_wsk_domain.build_profile_hmm("profile_hmm/hmm/WSK_v2.hmm", msa_filename)
# search for the binding region
hmm_for_wsk_domain.search_binding_regions("profile_hmm/WSK_v2.sto","profile_hmm/hmm/WSK_v2.hmm",seq_filename)
wsk_v2_hmm_hits = read_sto_files("profile_hmm/WSK_v2.sto")

In [9]:
wsk_hits_counter_v2 = dict(Counter([specie_name.rsplit('-')[0] for specie_name in wsk_v2_hmm_hits.keys()]))
wsk_hits_counter_v2 == wsk_hits_counter

True

The model still fails to detect the WAK motifs in zebrafinch and chicken, however, it does find the binding region in anole lizard which also has the WAK motif, could this be due to high conservation of the seed sequences?

## Investigating the WSK motif variation in bird and lizards

In [10]:
akap5_human = {fasta.id:str(fasta.seq) for fasta in SeqIO.parse(
    open("fasta_files/Homo_sapiens_AKAP5_sequence.fa"),
    'fasta')}

In [11]:
# # already in directory
# for taxa, taxa_id in {'birds':'8782', 'reptiles': '8504', 'amphibians':'8292'}.items():
#     result_temp_handle = NCBIWWW.qblast("blastp", "nr",
#                                         akap5_human['AKAP5-202'],
#                                         entrez_query = 'txid' + taxa_id + '[ORGN]', 
#                                         hitlist_size=500)
#     with open("blast_results/" + str(taxa) + ".xml", 'w') as f:
#         f.write(result_temp_handle.getvalue())

Since some of the blast hits sequences were very short I looked for the complete sequences and manually curated a fasta file with all AKAP75 hits sequences.

### 1. Birds

In [12]:
birds_seq_filename = 'fasta_files/Birds_AKAP5_seqs_manually_curated.fa'
birds_msa_filename = 'fasta_files/msa/Birds_AKAP5_seqs_manually_curated_msa.fa'
birds_AKAP79_WSK_domain = binding_regions()
#birds_AKAP79_WSK_domain.muscle_msa(birds_seq_filename, birds_msa_filename)
birds_akap5_seqs = birds_AKAP79_WSK_domain.parse_fasta_file(birds_seq_filename)
birds_akap5_msa = birds_AKAP79_WSK_domain.parse_fasta_file(birds_msa_filename)

In [13]:
ref_bird = "gallus_gallus"
# presumed calmodulin binding region in reference organism 
bird_base_string_wsk = 'RGAWAALKSLTKPRRGQKSSSRKKVSSDSQV'
start, end = birds_AKAP79_WSK_domain.find_binding_region(bird_base_string_wsk, ref_bird, birds_akap5_msa)
# WAK motif - is the variation in the motif an artifact? or can we also
# find it in species of the same clade?
start_m, end_m = birds_AKAP79_WSK_domain.find_binding_region('WAALK', ref_bird, birds_akap5_msa)
bird_wsk_residues_idx = (start_m,start_m+2,end_m-1)
filename = 'fasta_files/binding_regions/AKAP5_birds_WSK.fa'
birds_regions_dict, birds_aln = seq_domain_alignment(birds_akap5_msa,
                                                     birds_akap5_seqs,
                                                     start, end,
                                                     filename,
                                                     motif_coords=bird_wsk_residues_idx)

In [14]:
alv.view(birds_aln)

apteryx_ro RGAWAALKSLARPRRRGGACSRKKAPSASRA
anseranas_ RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
anas_platy RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
aythya_ful RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
asarcornis RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
oxyura_jam RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
anser_cygn RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
cygnus_atr RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
cygnus_olo RGAWAAIKSLTRPRRRQKSSSRKKVPSDSQV
alectura_l RGAWAAIKSLAKPRRRQKSSS-QKVPSDSQV
penelope_p RGAWAAIKSLTKPRRRQKSSSRKKVTSDSQV
coturnix_j RGAWAALKSLTKPRRGQKSSSRKKVSSDSQV
numida_mel RGAWAALKSFTKPRRRQKSSSRKNVSSDSQV
gallus_gal RGAWAALKSLTKPRRGQKSSSRKKVSSDSQV
phasianus_ RGAWAALKSLTKPRRGQKSSSRKKVSSDSQV
meleagris_ RGAWAALKSLTKPRRGQKSSSRKKVSSDSQV
centrocerc RGAWAAIKSLTKPRRGQKSSSRKKVSSDSQV
lagopus_le RGAWAALKSLTKPRRGQKSSSRKKVSSDSQV
upupa_epop KGAWAAIKNLTRPRRRQKSLSWKKVSSDSQV
rhinopomas KGAWTAIKNLTRPRRRQKPSSQKKVPSDSQV
calidris_p KGAWAAIKNLARPRRRQKSSSRKKVPSDSQV
erythrocer KGTWAAIKNLAKPWRRQKSSSRKKVPSDSQV
ptilonorhy KGAWAAIKNLARPRRRQTSSSRKKVPSDSQV
climacteri 

### 2. Reptiles

In [15]:
rept_seq_filename = 'fasta_files/Reptiles_AKAP5_seqs_manually_curated.fa'
rept_msa_filename = 'fasta_files/msa/Reptiles_AKAP5_seqs_manually_curated_msa.fa'
rept_AKAP79_WSK_domain = binding_regions()
#rept_AKAP79_WSK_domain.muscle_msa(rept_seq_filename, rept_msa_filename)
rept_akap5_seqs = rept_AKAP79_WSK_domain.parse_fasta_file(rept_seq_filename)
rept_akap5_msa = rept_AKAP79_WSK_domain.parse_fasta_file(rept_msa_filename)

In [16]:
ref_rept = "anolis_carolinensis"
# presumed calmodulin binding region in reference organism 
rept_base_string_wsk = 'GGAWLAFKRLVTSRRRSKSVLKKQQQSGGSRV'
start, end = rept_AKAP79_WSK_domain.find_binding_region(rept_base_string_wsk, ref_rept, rept_akap5_msa)
# WAK motif - is the variation in the motif an artifact? or can we also
# find it in species of the same clade?
start_m, end_m = rept_AKAP79_WSK_domain.find_binding_region('WLAFK', ref_rept, rept_akap5_msa)
rept_wsk_residues_idx = (start_m,start_m+2,end_m-1)
filename = 'fasta_files/binding_regions/AKAP5_rept_WSK.fa'
rept_regions_dict, rept_aln = seq_domain_alignment(rept_akap5_msa,
                                                     rept_akap5_seqs,
                                                     start, end,
                                                     filename,
                                                     motif_coords=rept_wsk_residues_idx)

In [17]:
alv.view(rept_aln)

gekko_japo GGAWLAFKRLVTLRRRSKFALKKEVQLG-SQV
varanus_ko GGAWVAFKRFVTLRRRSRSALKKQAQSG-SRV
python_biv GGAWLAFKRLVTLRRRSKSTWKKQTQVY-SRV
notechis_s GEAWLAFKRLVTLRRRSKSSLKKQTQAC-SRV
pseudonaja GEAWLAFKRLVTLRRRSKSSLKKQTQAC-SRV
naja_naja  GEAWLAFKRLVTLRRRSKSSLKKQTQAC-SRV
ophiophagu GEAWMAFKRLVTLRRRSKSSLKKQTQAC-SRV
protobothr GGAWLAFKRLVTLRRRSKSTLKKQTEAC-SRV
crotalus_t GGAWLAFKRLVTLRRRSKSTLKKQTEAY-SRV
pantheroph GGAWLAFKRLVTLRRRSKSSLKKQTQAC-SRV
thamnophis GGAWLAFKRLVTLRRRSKSSLKKQTQAC-SRV
thamnophis GGAWLAFKRLVTLRRRSKSSLKKQTQAC-SRV
pogona_vit ---WFAFKRLVTLRRRSRSSLKKPTQPG-SSV
zootoca_vi GGPWLSFKRLVTLRRRSKSAWKKEAQSG-SRV
lacerta_ag GGAWLSFKRLVTLRRRSKPALKKQAQSG-SRV
podarcis_m GGAWLSFKRLVTLRRRSKSALKKQAQSG-SRV
sceloporus GGAWLAFKRLVTSRRRSKSALKKQSQSRDSRV
anolis_car GGAWLAFKRLVTSRRRSKSVLKKQQQSGGSRV
          0↑                 20↑
